In [ ]:
#This is taken from work of Dan Lantos
#https://www.advancinganalytics.co.uk/blog/2021/07/05/facebook-prophet-and-the-stock-market-part-1
#   Facebook Prophet and the Stock Market (Part 1)

In [ ]:
# Pandas for pandas dataframe operations.
import pandas as pd
# NumPy package for numeric operations.
import numpy as np
# Prophet package used for forecasting.
from fbprophet import Prophet
# Prophet diagnostics for rolling cross-validation.
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
# yfinance package used to import dataset.
import yfinance as yf
# Datetime package used for date functions.
from datetime import datetime, timedelta
# Matplotlib package used for altering default plots.
import matplotlib.pyplot as plt
import matplotlib.lines as lines
%matplotlib inline  
# Plotly package used for candlestick charts.
import plotly.graph_objects as go


In [ ]:
# Logging package used to remove logging output.
import logging
# Command to remove logging messages from Prophet calls.
logging.getLogger("py4j").setLevel(logging.ERROR)

In [ ]:
# Select up to yesterday's close - as "close" today is current price if the market hasn't closed.
today = datetime.now() - timedelta( days = 1 )

In [ ]:
# Pull information for the FTSE100 index "ticker".
ftse = yf.Ticker( "^NSEI" )
ftse_df = ftse.history( start = '2016-01-01', end = today )

In [ ]:
# Inspect the dataframe.
ftse_df.head()

In [ ]:
# Reset the index to use it as our x variable.
ftse_df.reset_index( inplace=True )
# Create a candlestick chart of the dataset using plotly.graph_objects.
candlestick = go.Figure( data = [go.Candlestick( x = ftse_df['Date'],
                                                 open = ftse_df['Open'],
                                                 high = ftse_df['High'],
                                                 low = ftse_df['Low'],
                                                 close = ftse_df['Close']
                                               ) ] )

In [ ]:
# Show the figure.
candlestick.update_xaxes( title_text = 'Date' )
candlestick.update_yaxes( title_text = 'FTSE 100 Index' )
candlestick.show()


In [ ]:
# Select only the 2 columns we want, and rename them appropriately to be passed to Facebook Prophet.
ftse_prophet = ftse_df[['Date','Close']].rename( columns = {'Date':'ds','Close': 'y'} )

In [ ]:
# Inspect the new dataframe.
ftse_prophet.head()

In [ ]:
# Specify a cutoff day range.
days = 60

# Create a cutoff date using the days value.
today = datetime.now()
cutoff_date = today - timedelta( days )

In [ ]:
# Use cutoff_date to split our dataset to history and actuals, which we will use to validate the model. 
history_df = ftse_prophet[ ftse_prophet['ds'] <= cutoff_date ]
actuals_df = ftse_prophet[ ftse_prophet['ds'] > cutoff_date ]

In [ ]:
# Define the model - passing no hyperparameters results in a default model being created.
# This is bad practice in reality, but great for showing how simple Prophet is "out of the box".
model = Prophet()

# Fit the model to our history dataset.
model.fit( history_df )

In [ ]:
# Create a future dataframe using Prophet's functon make_future_dataframe.
# Remove any non-trading (or similar days) not found in the base dataframe.
future_df = model.make_future_dataframe( periods = days, freq ='d', include_history = True )
future_df = future_df[ future_df['ds'].isin( ftse_prophet['ds'] ) ]

In [ ]:
# Use the model to predict values for our test dataset.
forecast_df = model.predict( future_df )

# Plot the predictions, and overlay our actuals.
fig = model.plot( forecast_df )
ax = fig.gca()
ax.plot( actuals_df["ds"], actuals_df["y"], 'k.', color = "r" )
ax.set_xlim( [ datetime( 2019, 1, 1 ), today ] )

In [ ]:
# Define a function to calculate the Mean Absolute Percentage Error (MAPE) - user friendly error metric.
def mape( actuals, forecast ):
  x = 0
  for i in range( actuals.index.min(), actuals.index.max()+1 ):
    x += np.abs( ( actuals[i] - forecast[i] ) / actuals[i] )
  return x / len( actuals )

In [ ]:
# Ensure out dataframes have only corresponding entries.
forecast = forecast_df[ forecast_df['ds'].isin( actuals_df['ds'] ) ]
actuals = actuals_df[ actuals_df['ds'].isin( forecast['ds'] ) ]
print(type(forecast))
# Use our MAPE function to evaluate the success of our 60 day forecast.
mape_res = round( 100 * mape( actuals['y'], forecast['yhat'] ), 2 )
print( 'Forecast MAPE: ', mape_res )

In [ ]:
# Apply cross-validation on our model. This creates a forecast, for 180 days ahead, every 90 days, with 3 years of initial training data.
# As we have 5.5 years of data, this results in 8 forecasts (1 every quarter of a year, starting from 3 years -> 5 years).
crossv_df = cross_validation( model, initial = '1095 days', period = '90 days', horizon = '180 days' )

perf_df = performance_metrics( crossv_df )

# Use prophet's plot_cross_validation_metric to visualise the MAPE as the horizon increases.
fig = plot_cross_validation_metric( crossv_df, metric = 'mape')#, color='red' )

# Evaluate the mean MAPE for our forecasts.
crossv_mape = round( 100 * perf_df['mape'].mean(), 2 )
print( f'Cross validation MAPE: %s' % crossv_mape )